# Automatically generating object masks with SAM

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/automatic_mask_generator.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/automatic_mask_generator.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/automatic_mask_generator.ipynb)

This notebook shows how to segment objects from an image using the Segment Anything Model (SAM) with a few lines of code. 

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

The notebook is adapted from [segment-anything/notebooks/automatic_mask_generator_example.ipynb](https://github.com/opengeos/segment-anything/blob/pypi/notebooks/automatic_mask_generator_example.ipynb), but I have made it much easier to save the segmentation results and visualize them.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
# %pip install segment-geospatial leafmap localtileserver

In [ ]:
import os
import leafmap
from samgeo import SamGeo, show_image, download_file, overlay_images, tms_to_geotiff

## Create an interactive map

In [ ]:
m = leafmap.Map(center=[37.8713, -122.2580], zoom=17, height='800px')
m.add_basemap('SATELLITE')
m


Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [ ]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

## Download a sample image

In [ ]:
image = 'satellite.tif'
tms_to_geotiff(output=image, bbox=bbox, zoom=17, source='Satellite', overwrite=True)

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

In [ ]:
m.layers[-1].visible = False
m.add_raster(image, layer_name='Image')
m

## Initialize SAM class

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [ ]:
sam = SamGeo(
    model_type='vit_h',
    checkpoint=checkpoint,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

## Automatic mask generation

In [ ]:
sam.generate(image, output='masks.jpg', foreground=True)

In [ ]:
sam.show_masks(cmap='binary_r')

In [ ]:
sam.show_anns(axis='off', opacity=0.35, output='annotations.jpg')

## Automatic mask generation options

There are several tunable parameters in automatic mask generation that control how densely points are sampled and what the thresholds are for removing low quality or duplicate masks. Additionally, generation can be automatically run on crops of the image to get improved performance on smaller objects, and post-processing can remove stray pixels and holes. Here is an example configuration that samples more masks:

In [ ]:
sam_kwargs = {
    'points_per_side': 32,
    'pred_iou_thresh': 0.86,
    'stability_score_thresh': 0.92,
    'crop_n_layers': 1,
    'crop_n_points_downscale_factor': 2,
    'min_mask_region_area': 100,
}

In [ ]:
sam = SamGeo(
    model_type='vit_h',
    checkpoint=checkpoint,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=sam_kwargs,
)

In [ ]:
sam.generate(image, output='masks2.jpg', foreground=True)

In [ ]:
sam.show_masks(cmap='binary_r')

In [ ]:
sam.show_anns(axis='off', opacity=1, output='annotations2.jpg')

Overlay the annotations on the image and use the slider to change the opacity interactively.

In [ ]:
overlay_images(image, 'annotations2.jpg', backend='TkAgg')

![](https://i.imgur.com/I1IhDgz.gif)